In [1]:
from pymodbus.client import ModbusTcpClient

In [2]:
ip = '192.168.0.113'
port = 502

register_address_map = {"TEMP_PV": 0,
                        "RH_PV": 2,
                        "RH_SV_Read_Only": 3,
                        "Program_Status": 4,
                        "TEMP_SV": 7,
                        "RH_SV_Read_Write": 16
                        } 
coil_address_map = {"Profile_Run": 0,
                    "Lamp_Command": 1,
                    "Manual_Run" : 2,
                    "Batch_Start_Flag": 3
                   }

In [3]:
# Initialize modbus connection
client = ModbusTcpClient(host=ip, port=port)
connection = client.connect()

Connection to (192.168.0.113, 502) failed: timed out


In [32]:
print("\nReading coils:")
for key, value in coil_address_map.items():
    rd = client.read_coils(value)
    print()
    print(rd)
    print(f"{key: <10}: {rd.bits}")


Reading coils:

ReadCoilsResponse(8)
Profile_Run: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Lamp_Command: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Manual_Run: [False, False, False, False, False, False, False, False]

ReadCoilsResponse(8)
Batch_Start_Flag: [False, False, False, False, False, False, False, False]


In [12]:
client.write_coil(1, False)

In [19]:
rd = client.read_holding_registers(16, 2)
print()
assert(rd.function_code < 0x80)     # test that we are not an error
print(rd)
print(rd.registers)


ReadHoldingRegistersResponse (2)
[0, 17008]


In [20]:
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder

registers = rd.registers
# decoder = BinaryPayloadDecoder.fromRegisters(registers, wordorder=Endian.LITTLE)
decoder = BinaryPayloadDecoder.fromRegisters(registers, Endian.BIG, wordorder=Endian.LITTLE)
print ("Result: " + str(decoder.decode_32bit_float()))

Result: 60.0


90 17076,
50 16968, 
10 16672

In [125]:
rd.registers[1]*100/65535

25.995269703212024

In [7]:
rq = client.write_register(16, 70)       # RH_SV_Read_Write = 70 %
rd = client.read_holding_registers(value)
print(rd)

ReadHoldingRegistersResponse (1)


In [8]:
print("\nReading holding registers:")
for key, value in register_address_map.items():
    rd = client.read_holding_registers(value)
    print()
    print(rd)
    print(f"{key: <10}: {rd.registers}")


Reading holding registers:

ReadHoldingRegistersResponse (1)
TEMP_PV   : [249]

ReadHoldingRegistersResponse (1)
RH_PV     : [89]

ReadHoldingRegistersResponse (1)
RH_SV_Read_Only: [60]

ReadHoldingRegistersResponse (1)
Program_Status: [0]

ReadHoldingRegistersResponse (1)
TEMP_SV   : [230]

Exception Response(131, 3, IllegalAddress)
RH_SV_Read_Write: []


In [9]:
print("\nWriting to set value holding registers:\n")

key = "TEMP_SV"
value = register_address_map[key]
rq = client.write_register(value, 230)       # TEMP_SV = 20.0 deg C
rd = client.read_holding_registers(value)
print(rd)
print(f"{key: <10}: {rd.registers}")

key = "RH_SV_Read_Write"
value = register_address_map[key]
rq = client.write_register(value, 70)       # RH_SV_Read_Write = 70 %
rd = client.read_holding_registers(value)
print(rd)
print(f"{key: <10}: {rd.registers}")


Writing to set value holding registers:

ReadHoldingRegistersResponse (1)
TEMP_SV   : [230]
Exception Response(131, 3, IllegalAddress)
RH_SV_Read_Write: []


In [9]:
# Disconnect device
client.close()

In [10]:
key = "TEMP_SV"
value = register_address_map[key]
rq = client.write_register(value, 230)       # TEMP_SV = 20.0 deg C
rd = client.read_holding_registers(value)
print(rd)
print(f"{key: <10}: {rd.registers}")

ReadHoldingRegistersResponse (1)
TEMP_SV   : [230]


In [7]:
key = "Program_Status"
value = register_address_map[key]
rd=client.read_holding_registers(value)
rd.registers

[0]

In [14]:
client.write_coil(3, True)

In [16]:
client.write_coil(0, True)

In [10]:
key = "TEMP_PV"
value = register_address_map[key]
rd = client.read_holding_registers(value)
print(rd.registers)

Connection to (192.168.0.113, 502) failed: timed out


ConnectionException: Modbus Error: [Connection] Failed to connect[ModbusTcpClient 192.168.0.113:502]